In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import os


c:\Users\filip\dev\malware-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Paths
dataset_path = "./data/csic_database_labeled.csv"
output_dir = "./codebert_finetuned_csic"

# Load dataset
df = pd.read_csv(dataset_path)

# Verify required columns
required_columns = ['llm_input', 'label']
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"Dataset must contain columns: {required_columns}")


In [3]:

# Split dataset: 80% train, 10% validation, 10% test
train_val, test = train_test_split(df, test_size=0.1, random_state=42, stratify=df['label'])
train, val = train_test_split(train_val, test_size=0.1111, random_state=42, stratify=train_val['label'])
# Results in ~80% train, ~10% val, ~10% test (0.1111 = 10/90)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train[['llm_input', 'label']])
val_dataset = Dataset.from_pandas(val[['llm_input', 'label']])
test_dataset = Dataset.from_pandas(test[['llm_input', 'label']])

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

# Tokenize function
def tokenize_function(examples):
    return tokenizer(
        examples['llm_input'],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )


In [4]:

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


Map: 100%|██████████| 6107/6107 [00:00<00:00, 9470.18 examples/s]


In [6]:
import wandb

# Initialize WandB
wandb.init(project="codebert-csic", name="cuda-4070ti", reinit=True)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/codebert-base", num_labels=2
)
device = "cuda"
model.to(device)


# Define compute_metrics function for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,  # Effective batch size = 16 * 2 = 32
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=100,
    seed=42,
    fp16=True,  # Mixed precision for CUDA
    dataloader_num_workers=4,  # Parallel data loading
    report_to="wandb",  # Enable WandB logging
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train model
print("Starting fine-tuning...")
trainer.train()

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Starting fine-tuning...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.029200,0.018721,0.994432,0.998387,0.988029,0.993181
2,0.014200,0.010042,0.997543,0.998799,0.995211,0.997002
3,0.004800,0.009176,0.997543,0.998001,0.996010,0.997004


TrainOutput(global_step=4581, training_loss=0.054700490618293345, metrics={'train_runtime': 1199.3666, 'train_samples_per_second': 122.194, 'train_steps_per_second': 3.82, 'total_flos': 3.856050382934016e+16, 'train_loss': 0.054700490618293345, 'epoch': 3.0})

In [8]:

# Evaluate on test set
print("\nEvaluating on test set...")
test_results = trainer.evaluate(test_dataset)
print("Test set results:")
for metric, value in test_results.items():
    print(f"{metric}: {value:.4f}")

# Save model and tokenizer
print(f"\nSaving model to {output_dir}")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


Evaluating on test set...
Test set results:
eval_loss: 0.0132
eval_accuracy: 0.9972
eval_precision: 0.9996
eval_recall: 0.9936
eval_f1: 0.9966

Saving model to ./codebert_finetuned_csic


('./codebert_finetuned_csic\\tokenizer_config.json',
 './codebert_finetuned_csic\\special_tokens_map.json',
 './codebert_finetuned_csic\\vocab.json',
 './codebert_finetuned_csic\\merges.txt',
 './codebert_finetuned_csic\\added_tokens.json',
 './codebert_finetuned_csic\\tokenizer.json')

In [9]:

# Example inference on a single request
sample_input = "Method: POST\nURL: /tienda1/login.jsp\nContent: username=admin' OR '1'='1'-- &password=anything"
sample_tokenized = tokenizer(sample_input, return_tensors="pt", max_length=512, truncation=True, padding=True)
model.eval()
with torch.no_grad():
    outputs = model(**{k: v.to(model.device) for k, v in sample_tokenized.items()})
    predicted_label = outputs.logits.argmax().item()
print(f"\nSample input: {sample_input}")
print(f"Predicted label: {'Normal' if predicted_label == 0 else 'Anomalous'}")


Sample input: Method: POST
URL: /tienda1/login.jsp
Content: username=admin' OR '1'='1'-- &password=anything
Predicted label: Anomalous
